# Calculating trigger times

Import modules

In [1]:
%matplotlib inline
import pandas as pd
import os, glob
import matplotlib.backends.backend_pdf
import matplotlib.pyplot as plt
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import scipy.stats as stats
from scipy.stats import zscore
from scipy.signal import savgol_filter
import matplotlib.gridspec as gridspec

Read compilation table

In [2]:
df = pd.read_csv(r'C:\Users\jlh94\FORCE PLATE SOFTWARE (21Nov2020)\JHS4_Raw_Data\JHS4_BlockAveragesCompilation(20.0Hz).csv')

Set up data

In [12]:
participantNumber = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                     11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                     21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
                     31, 32, 33, 34, 35]

blockNumber = [1, 2, 3, 4]

freq = 20 # in Hz
trialLen = 4 # in seconds 
numTrials = 30

timeInterval = (trialLen*freq)
blockInterval = timeInterval*numTrials # 20Hz for 4 seconds for 30 trials

maxSampleRejection = 25 #value is percent
minimumSample = timeInterval-((timeInterval/100)*maxSampleRejection)

rejectTrigger = 903
acceptTrigger = 72

Keep only relevent columns

In [4]:
df = df[['Study_Block', 'Participant', 'Time (s)','Trigger', 'FzLocallyFinite', 'X_Corr (mm)']]


Create empty dataframe where calculated data will be appended

In [13]:
dfTriggerRelTime = pd.DataFrame()

Loop through data to identify x co-ordinate where new trigger will be

In [14]:
for p in participantNumber:
    dfP = df[df['Participant'] == p]  # when value in 'Participant' column is equal to the participant value in loop
    for b in blockNumber:
        dfB = dfP[dfP['Study_Block'] == b].reset_index(drop=True) # when value in 'Study_block' column is equal to the block value in loop
        num =[] # create empty array
        for ind in dfB.index: #loop through the data frame 
            if np.asarray(dfB.loc[ind,['Trigger']]) > 1.3: # if the value in the 'Trigger' column is greater than 1.3
                num.append(ind) # appened the index value to the empty array that was previously created
        
        
        dfT= dfB[num[0]:num[0]+blockInterval] # slice the data from the first trigger to the end of the block
        triggerRelativeTime = pd.DataFrame(columns=['Study_Block', 'Participant', 'Time (s)', 'Relative Time', 'Trigger',
                                'X_Corr (mm)']) # create an empty dataframe where we will append calculated data
        
        c=0  # create a counter starting at 0
        
        for t in range(30): # loop through trials
            dfEpoch= dfT[c:c+timeInterval].reset_index(drop=True).abs() #epoch the data and change value to absolute
            c = c+timeInterval #add the time to c so that it the epoch window moves
            dfEpoch.insert(3, "Relative Time", np.arange(start=0, stop=4, step=0.05), True) # insert a column with the relative time from the trigger
            dfFiltered = dfEpoch[dfEpoch['FzLocallyFinite'] == 1] # only look at the rows where 'FzLocallyFinite' is equal to 1, not 0 
            
            #The loop below rejects trials where there are fewer than 75% of samples 
            if ((len(dfFiltered)) < minimumSample): #skip over dataframes that have less than 75% of samples
                triggerRelativeTime = triggerRelativeTime.append({'Study_Block': b, 'Participant': p, 'Time (s)': np.nan,
                                                     'Relative Time': np.nan, 'Trigger': rejectTrigger, 'X_Corr (mm)':np.nan},ignore_index=True)        
            else:
                newTriggerIndex= np.asarray(dfFiltered[['X_Corr (mm)']].idxmin())
                triggerData = dfEpoch.loc[newTriggerIndex]
                triggerRelativeTime = triggerRelativeTime.append({'Study_Block': b, 'Participant': p, 'Time (s)': float(triggerData['Time (s)']),
                                                     'Relative Time': float(triggerData['Relative Time']), 'Trigger': acceptTrigger, 'X_Corr (mm)': float(triggerData['X_Corr (mm)'])},ignore_index=True)
                 
    ##### Remove trials based on zscore
        SD=2
        
        triggerRelativeTime['Relative Time zscore'] = stats.zscore(triggerRelativeTime['Relative Time'])
        triggerRelativeTime = triggerRelativeTime.abs()
        triggerRelativeTime.loc[triggerRelativeTime['Relative Time zscore'] >= SD, 'Relative Time zscore'] = np.nan

        triggerRelativeTime['Trigger'] = np.where(triggerRelativeTime['Relative Time zscore'].isnull(), rejectTrigger, acceptTrigger) # when the 'Relative Time zscore' is nan change the trigger value to the rejected trigger number, else keep the accepted trigger value
        
        dfTriggerRelTime = dfTriggerRelTime.append(triggerRelativeTime, ignore_index=True) #append data to datafram outside the loop

In [15]:
exportdfTriggerRelTime= dfTriggerRelTime.to_csv(r'C:\Users\jlh94\FORCE PLATE SOFTWARE (21Nov2020)\JHS4_Raw_Data\triggers.csv', index=None)

